In [ ]:
SRC = "//192.168.0.1/HDD2/pub/files/movies/CaroleAndTuesday"
FFMPEG_EXE = "c:/Users/shining/Desktop/ffmpeg/bin/ffmpeg.exe"
MOV_EXT = "mkv"
SRC_ENCODING = 'euc-kr'
WORKING = SRC+"/working"
W_SMI = WORKING+"/smi"
W_MP4 = WORKING+"/mp4"

import os
SMIs = sorted([SRC+"/"+f for f in filter(lambda f:f.endswith(".smi"), os.listdir(SRC))])
MOVs = sorted([SRC+"/"+f for f in filter(lambda f:f.endswith("."+MOV_EXT), os.listdir(SRC))])

def run_cmd(cmd):
    import os
    return os.popen(cmd).readlines()

def ffmpeg(options):
    run_cmd("%s %s"%(FFMPEG_EXE, options))
    
def iconv(src_path, org_encoding, tgt_path, dst_encoding='utf-8'):
    with open(src_path, 'r', encoding=org_encoding) as f:
        s = f.read()
    with open(tgt_path, 'w', encoding=dst_encoding) as f:
        f.write(s)
    return tgt_path
        
def iconv_all(src_list, org_encoding, tgt_pattern, dst_encoding='utf-8'):
    return [iconv(s, org_encoding, tgt_pattern%(i+1), dst_encoding) for i, s in enumerate(src_list)]
    
def to_srt(smi_path):
    ffmpeg("-i %s %s"%(smi_path, smi_path.replace(".smi", ".srt")))
    return smi_path.replace(".smi", ".srt")

def encode_with_subtitle(mov, srt, mp4):
    ffmpeg('-i "%s" -vf "subtitles=\'%s\':force_style=\'FontName=맑은 고딕,Fontsize=30\'" -c:V libx264 -c:a aac "%s"'
          %(mov, srt, mp4))
    return mp4

run_cmd("powershell mkdir -p %s"%W_SMI)
run_cmd("powershell mkdir -p %s"%W_MP4)

[to_srt(s) for s in iconv_all(SMIs, 'euc-kr', W_SMI+"/%02d.smi")]

for i, m in enumerate(MOVs):
    srt = W_SMI + "/%02d.srt"%(i+1)
    mp4 = W_MP4 + "/%02d.mp4"%(i+1)
    print(encode_with_subtitle(m, srt, mp4))

"done"